In [1]:
from sklearn.neighbors import KNeighborsClassifier
# from sklearn import preprocessing
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
import os
from flask import Flask, flash, request, redirect, url_for, render_template

app = Flask(__name__)
upload_path = 'inputs'
app.config['UPLOAD_FOLDER'] = upload_path

In [3]:
@app.route("/")#methods=["GET", "POST"]
def main():
    return render_template('inputForm.html')

@app.route("/get-file", methods=['POST', 'GET'])
def submitted():
    global file_content
    
    if request.method == 'POST':
        result = request.form['file-select']
        
        result = os.path.join(upload_path, result)
        
#         text = open(result,'r+')
#         content = text.read()
#         file_content = result
#         text.close()
        
        listPerson = []
        listOrg = []
        listLocation = []
        
        listPerson, listOrg, listLocation = prediksi(result)
        
#         listPerson = ['Airlangga', 'Hartarto', 'Kevin']
#         listOrg = ['ITHB', 'Kemendikbud', 'MK']
#         listLocation = ['Jakarta', 'Bandung', 'Istana negara']
        
#         print(content)
        return render_template('upload.html', list_person = listPerson, list_org = listOrg, list_loc = listLocation)

<h1>Load hasil baca berita</h1>

In [4]:
def loadPickle(name):
    file = open(name, 'rb')
    data = pickle.load(file)
    file.close()
    
    return data

In [5]:
def listOfDictToDataFrame(listOfDict):
    dataFrame = pd.DataFrame(listOfDict)
    
    x = dataFrame.iloc[:, :-1].values
    y = dataFrame.iloc[:, 4].values
    
    return dataFrame, x, y

<h1>Baca Data Test</h1>

In [6]:
def extFeature(words):
    listOfDictFeature = []
    i = 0
    for i in range(len(words)): #iterasi dari awal tiap kata
        dictFeature = {}
        
        dictFeature = inputToDict(words, i)
        
        listOfDictFeature.append(dictFeature)
        
    return listOfDictFeature

def inputToDict(words, i):
    dictFeature = {}
    
    dictFeature['index'] = i    
    dictFeature['first_upper_case'] = isFirstUpper(words[i])
    dictFeature['prev_word'] = isEdge(i, words)
    dictFeature['word'] = words[i]
    dictFeature['jenis'] = 'o'
    
    return dictFeature

def isFirstUpper(word):
    if word[0].isupper():
        return 'y'
    else:
        return 'n'
    
def isEdge(i, words):
    if i == 0:
        return 'null'
    else:
        return words[i-1]

In [7]:
def readPerWord(file):
    words = []
    f = open(file, 'r') #open file
    for line in f:
        words = line.split(' ') #tiap satu baris split per kata
    
    f.close()
    
    words[len(words)-1] = words[len(words)-1].replace('\n',' ')

    return words

In [8]:
def prediksi(file):
    testWord = readPerWord(file)
#     print(tesWord)
    listOfDictFeature = extFeature(testWord)
#     print(listOfDictFeature)
    panjangTestWord = len(listOfDictFeature)
    
    data = loadPickle("training.pickle")
    
    data.extend(listOfDictFeature)
    
    totalPanjangData = len(data)
    
    dataFrame, x, y = listOfDictToDataFrame(data)
    
#     for i in x:
#         print(i)
    
    forx = OneHotEncoder(handle_unknown='ignore')
    forx.fit(x)
    newx = forx.transform(x).toarray()
#     print(len(newx),len(newx[0]))
    
    X_train, X_test, y_train, y_test = train_test_split(newx, y, test_size = panjangTestWord, shuffle=False)
    
#     print(len(X_train),len(X_train[0]))
#     print(len(X_test),len(X_test[0]))
#     print(len(y_train))
#     print(len(y_test))
    
    classifier = KNeighborsClassifier(n_neighbors=5)
    classifier.fit(X_train,y_train)
    
    y_pred = classifier.predict(X_test)
#     print(y_pred)
    
    wordInvers = forx.inverse_transform(X_test)
#     print(wordInvers)
    
    person = []
    location = []
    organization = []
    for i in range(len(y_pred)):
        if y_pred[i] == 'person':
            person.append(wordInvers[i][3])
        elif y_pred[i] == 'location':
            location.append(wordInvers[i][3])
        elif y_pred[i] == 'organization':
            organization.append(wordInvers[i][3])
#     print(person)
#     print(location)
#     print(organization)
            
    return person, location, organization

In [9]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Nov/2019 11:42:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2019 11:43:07] "POST /get-file HTTP/1.1" 200 -
